In [1]:
import cv2
import numpy as np
import math

#Parameter sind der Frame und ein boolwert ob das Rechteck eingezeichnet werden soll oder nicht
def GetEyes(frame, rectangle):
    # Umwandeln in Graustufenbild
    grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Haar-Cascade-Datei für Augen laden
    eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
    # Erkennen der Augen
    eyes = eye_cascade.detectMultiScale(grayFrame, scaleFactor=1.1, minNeighbors=4)

    #Falsch erkannte Objekte herausfiltern die nicht in den möglichen Pixelbereich liegen können
    eyesFiltert = list()
    for i in range(len(eyes)):
        if eyes[i][3] > 50 and eyes[i][1] < 250 and eyes[i][1] > 150:
            eyesFiltert.append(eyes[i])
    #print(eyesFiltert)

    if rectangle:
        for (x, y, w, h) in eyesFiltert:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    #Augen Positionen herausfiltern (x,y) und sortieren von augen Position links nach rechts
    eyesPos = list()
    for i in range(len(eyesFiltert)):
            eyesPos.append((eyesFiltert[i][0],eyesFiltert[i][1]))
    return eyesPos

#Berechnen des Winkels zwischen den beiden Punkten        
def CalcAngleAndSpacing(cords):
    print("Cords der Augen: ",cords)
    deltaX = cords[0][0] - cords[1][0]
    deltaY = cords[0][1] - cords[1][1]
    print("delta X Augen:", deltaX, "delta Y Augen: ", deltaY)
    tan = deltaY/deltaX
    alpha = math.degrees(math.tan(tan)) * (-1)
    spacing = math.sqrt((deltaX ** 2 + deltaY ** 2))
    print("Abstand der Augen: ", spacing)
    return alpha, spacing

#Rotieren und Skalieren des Bildes mit berechnetem Winkel 
def RotateAndScale(image, angle, spacing, eyesPos):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    image = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)

    #Bild Sklaieren auf den gleichen Augenabstand des linken Bildes
    scale_percent = 100 - (1-spacing/60)*100 #scale des Bildes in % CutFace hat einen Abstand von 60
    print("Skalierung in Prozent", scale_percent)
    width = int(image.shape[1] * scale_percent / 100) #Berechung der Breite
    height = int(image.shape[0] * scale_percent / 100) #Berechnung der Höhe
    dim = (width, height) #Tupel aus neuer Skalierung
    result = cv2.resize(image, dim, interpolation = cv2.INTER_AREA) #Resize Methode
    cv2.imwrite("Result.png", result)

    #contouren der Alpha Kanalebene herausfinden und Punkte in contours speichern
    contours, hierarchy = cv2.findContours(result[:,:,3], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    #Min und Max herausfinden um kleinstes Rechteck zu berechnen in dem das Gesicht liegen kann
    right = np.max([np.max(contour[:,:,0]) for contour in contours])
    bottom = np.max([np.max(contour[:,:,1]) for contour in contours])
    left = np.min([np.min(contour[:,:,0]) for contour in contours])
    top = np.min([np.min(contour[:,:,1]) for contour in contours])

    #Image croppen das es genau so groß wie das Gesicht ist
    croppedImage = result[top:bottom, left:right]

    #Berechnen des Offsets der beim Überlagern erfolgen muss
    eyesCropped = GetEyes(croppedImage, False)
    
    try:
        xOffset = eyesPos[0][0] - eyesCropped[0][0]
        yOffset = eyesPos[0][1] - eyesCropped[0][1]
    except:
        xOffset = 0
        yOffset = 0

    cv2.imwrite("Cropped.png", croppedImage)

    return croppedImage, xOffset, yOffset



def Overlap(background , overlay, x, y):

    frame = background

    background_width = background.shape[1]
    background_height = background.shape[0]

    if x >= background_width or y >= background_height:
        return background

    h, w = overlay.shape[0], overlay.shape[1]

    if x + w > background_width:
        w = background_width - x
        overlay = overlay[:, :w]

    if y + h > background_height:
        h = background_height - y
        overlay = overlay[:h]

    if overlay.shape[2] < 4:
        overlay = np.concatenate(
            [
                overlay,
                np.ones((overlay.shape[0], overlay.shape[1], 1), dtype = overlay.dtype) * 255
            ],
            axis = 2,
        )

    overlay_image = overlay[..., :3]
    mask = overlay[..., 3:] / 255.0
    try:
        background[y:y+h, x:x+w] = (1.0 - mask) * background[y:y+h, x:x+w] + mask * overlay_image

    except:
        return frame

    return background

# Video öffnen
cap = cv2.VideoCapture('TestVid.mp4')

# Solange das Video läuft
while(cap.isOpened()):

    # Frame für Frame auslesen
    ret, frame = cap.read()

    # Wenn das Video beendet ist, Schleife abbrechen
    if not ret:
        break

    #Augen Position erfassen
    eyesPos = GetEyes(frame, True)

    #Berechnen des Winkels und Abstand der Augen
    if len(eyesPos) == 2:
        angle, spacing = CalcAngleAndSpacing(eyesPos)
        print("Winkel", angle)
        print("Abstand der Augen", spacing)
        face = cv2.imread('CutFace.png', cv2.IMREAD_UNCHANGED)
        #Gesicht Rotieren und auf den Augenabstad skalieren
        rotatet, xOffset, yOffset = RotateAndScale(face, angle, spacing, eyesPos)
        #Überlappen der beiden Bilder mit gegebenen Offset
        frame = Overlap(frame, rotatet, xOffset, yOffset)
        #Gausfilter
        frame = cv2.blur(frame,(3,3))
        
    #Anzeigen des Frames
    cv2.imshow('Pic', rotatet)
    cv2.imshow('Video', frame)

    # Auf Tastendruck warten, um das Fenster zu schließen
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Ressourcen freigeben
cap.release()
cv2.destroyAllWindows()


Cords der Augen:  [(214, 192), (313, 199)]
delta X Augen: -99 delta Y Augen:  -7
Abstand der Augen:  99.24716620639605
Winkel -4.057981600901857
Abstand der Augen 99.24716620639605
Skalierung in Prozent 165.41194367732675


QObject::moveToThread: Current thread (0x15cade0) is not the object's thread (0x22b82b0).
Cannot move to target thread (0x15cade0)

Qt: Session management error: None of the authentication protocols specified are supported
QObject::moveToThread: Current thread (0x15cade0) is not the object's thread (0x22b82b0).
Cannot move to target thread (0x15cade0)

QObject::moveToThread: Current thread (0x15cade0) is not the object's thread (0x22b82b0).
Cannot move to target thread (0x15cade0)

QObject::moveToThread: Current thread (0x15cade0) is not the object's thread (0x22b82b0).
Cannot move to target thread (0x15cade0)

QObject::moveToThread: Current thread (0x15cade0) is not the object's thread (0x22b82b0).
Cannot move to target thread (0x15cade0)

QObject::moveToThread: Current thread (0x15cade0) is not the object's thread (0x22b82b0).
Cannot move to target thread (0x15cade0)

QObject::moveToThread: Current thread (0x15cade0) is not the object's thread (0x22b82b0).
Cannot move to target threa

Cords der Augen:  [(214, 193), (312, 199)]
delta X Augen: -98 delta Y Augen:  -6
Abstand der Augen:  98.18350166906862
Winkel -3.5122945047903285
Abstand der Augen 98.18350166906862
Skalierung in Prozent 163.6391694484477
Cords der Augen:  [(214, 194), (313, 199)]
delta X Augen: -99 delta Y Augen:  -5
Abstand der Augen:  99.12618221237011
Winkel -2.896189151487238
Abstand der Augen 99.12618221237011
Skalierung in Prozent 165.2103036872835
Cords der Augen:  [(213, 193), (312, 198)]
delta X Augen: -99 delta Y Augen:  -5
Abstand der Augen:  99.12618221237011
Winkel -2.896189151487238
Abstand der Augen 99.12618221237011
Skalierung in Prozent 165.2103036872835
Cords der Augen:  [(213, 195), (312, 199)]
delta X Augen: -99 delta Y Augen:  -4
Abstand der Augen:  99.08077512817509
Winkel -2.316241538621982
Abstand der Augen 99.08077512817509
Skalierung in Prozent 165.13462521362516
Cords der Augen:  [(214, 195), (311, 199)]
delta X Augen: -97 delta Y Augen:  -4
Abstand der Augen:  97.0824391947